In [45]:
from inventory.models import (
    Brand,
    Category,
    Product,
    Order
)
from django.contrib.auth.models import User

In [2]:
Product.objects.all()

<QuerySet [<Product: Men Relaxed Fit Mid Rise Blue Jeans>, <Product: Motorola Edge 50 Fusion (Marshmallow Blue, 256 GB)  (12 GB RAM)>, <Product: SAMSUNG 8 kg 5 star, Ecobubble, Digital Inverter, Fully Automatic Top Load Washing Machine Grey>]>

In [25]:
from django.db.models import (
    Count,
    F,
    Sum,
    OuterRef,
    Value,
    IntegerField,
    Prefetch,
    Q,
    ExpressionWrapper,
    functions,
    CharField,
    Case,
    When,
    Func
)

In [30]:
get_product_with_total_price = Order.objects.all().annotate(total_price = Product.objects.filter(uuid = OuterRef("product__uuid")).values("price")).annotate(total_price = F("quantity") * F("total_price"))

In [31]:
get_product_with_total_price

<QuerySet [<Order: Men Relaxed Fit Mid Rise Blue Jeans>, <Order: Motorola Edge 50 Fusion (Marshmallow Blue, 256 GB)  (12 GB RAM)>, <Order: SAMSUNG 8 kg 5 star, Ecobubble, Digital Inverter, Fully Automatic Top Load Washing Machine Grey>]>

In [32]:
from django_pandas.io import read_frame

In [33]:
df = read_frame(get_product_with_total_price)

In [44]:
df.loc[:, ['customer','product','quantity','total_price']]

,customer,product,quantity,total_price
0,Mayank,Men Relaxed Fit Mid Rise Blue Jeans,2,998.0
1,Mayank,"Motorola Edge 50 Fusion (Marshmallow Blue, 256...",1,24999.0
2,Deqode,"SAMSUNG 8 kg 5 star, Ecobubble, Digital Invert...",3,58470.0


In [46]:
get_customer_orders = User.objects.filter(username = "Mayank")

In [47]:
get_customer_orders

<QuerySet [<User: Mayank>]>

In [58]:
get_customer_orders = User.objects.filter(username = "Mayank").annotate(total_orders = Order.objects.filter()

In [60]:
get_customer_orders

OperationalError: sub-select returns 2 columns - expected 1